In [1]:
import numpy as np
import pickle
import pandas as pd
import datetime
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import performance_metrics

prophet_final = pickle.load(open('Models/Prophet_Final.pkl', 'rb'))
df = pd.read_pickle("Data/Prophet_Forecast_DF.pkl")

In [2]:
def nutritional_breakdown(df, converted_data):

    total_calories = converted_data['Daily_Calories']
    percent_fat = converted_data['Daily_Fat']
    percent_protein = converted_data['Daily_Protein']
    percent_carb = converted_data['Daily_Carbs']
    percent_breakfast = converted_data['Percent_Breakfast']
    percent_lunch = converted_data['Percent_Lunch']
    percent_dinner = converted_data['Percent_Dinner']

    nutrient_breakdown = {
        'Calories_Breakfast': np.mean(df['Calories_Breakfast']),
        'Saturated_Fat_Breakfast': np.mean(df['Saturated_Fat_Breakfast']),
        'Cholesterol_Breakfast': np.mean(df['Cholesterol_Breakfast']),
        'Polyunsaturated_Fat_Breakfast': np.mean(df['Polyunsaturated_Fat_Breakfast']),
        'Monounsaturated_Fat_Breakfast': np.mean(df['Monounsaturated_Fat_Breakfast']),
        'Trans_Fat_Breakfast': np.mean(df['Trans_Fat_Breakfast']),
        'Carbohydrates_(g)_Breakfast': np.mean(df['Carbohydrates_(g)_Breakfast']),
        'Sugar_Breakfast': np.mean(df['Sugar_Breakfast']),
        'Sodium_(mg)_Breakfast': np.mean(df['Sodium_(mg)_Breakfast']),
        'Protein_(g)_Breakfast': np.mean(df['Protein_(g)_Breakfast']),
        'Calories_Lunch': np.mean(df['Calories_Lunch']),
        'Saturated_Fat_Lunch': np.mean(df['Saturated_Fat_Lunch']),
        'Polyunsaturated_Fat_Lunch': np.mean(df['Polyunsaturated_Fat_Lunch']),
        'Monounsaturated_Fat_Lunch': np.mean(df['Monounsaturated_Fat_Lunch']),
        'Trans_Fat_Lunch': np.mean(df['Trans_Fat_Lunch']),
        'Cholesterol_Lunch': np.mean(df['Cholesterol_Lunch']),
        'Sodium_(mg)_Lunch': np.mean(df['Sodium_(mg)_Lunch']),
        'Carbohydrates_(g)_Lunch': np.mean(df['Carbohydrates_(g)_Lunch']),
        'Sugar_Lunch': np.mean(df['Sugar_Lunch']),
        'Protein_(g)_Lunch': np.mean(df['Protein_(g)_Lunch']),
        'Calories_Dinner': np.mean(df['Calories_Dinner']),
        'Saturated_Fat_Dinner': np.mean(df['Saturated_Fat_Dinner']),
        'Polyunsaturated_Fat_Dinner': np.mean(df['Polyunsaturated_Fat_Dinner']),
        'Monounsaturated_Fat_Dinner': np.mean(df['Monounsaturated_Fat_Dinner']),
        'Trans_Fat_Dinner': np.mean(df['Trans_Fat_Dinner']),
        'Cholesterol_Dinner': np.mean(df['Cholesterol_Dinner']),
        'Sodium_(mg)_Dinner': np.mean(df['Sodium_(mg)_Dinner']),
        'Carbohydrates_(g)_Dinner': np.mean(df['Carbohydrates_(g)_Dinner']),
        'Sugar_Dinner': np.mean(df['Sugar_Dinner']),
        'Protein_(g)_Dinner': np.mean(df['Protein_(g)_Dinner'])
    }

    cals_per_g_fat = 9
    cals_per_g_protein = 4
    cals_per_g_carb = 4
    
    percent_breakfast = percent_breakfast / 100
    percent_lunch = percent_lunch / 100
    percent_dinner = percent_dinner / 100
    
    percent_fat = percent_fat / 100
    percent_protein = percent_protein / 100
    percent_carb = percent_carb / 100

    breakfast_multiplier = percent_breakfast / np.mean(df['Calories_Breakfast'] / df['Calories'])
    lunch_multiplier = percent_lunch / np.mean(df['Calories_Lunch'] / df['Calories'])
    dinner_multiplier = percent_dinner / np.mean(df['Calories_Dinner'] / df['Calories'])

    g_fat = total_calories * percent_fat / cals_per_g_fat
    g_protein = total_calories * percent_protein / cals_per_g_protein
    g_carbs = total_calories * percent_carb / cals_per_g_carb

    fat_multiplier = g_fat / np.mean(df['Fat (g)'])
    protein_multiplier = g_protein / np.mean(df['Protein (g)'])
    carb_multiplier = g_carbs / np.mean(df['Carbohydrates (g)'])

    fat_breakdown = {}

    fat_columns = ['Saturated_Fat_Breakfast', 'Cholesterol_Breakfast',
                   'Polyunsaturated_Fat_Breakfast', 'Monounsaturated_Fat_Breakfast',
                   'Trans_Fat_Breakfast', 'Sodium_(mg)_Breakfast', 'Saturated_Fat_Lunch',
                   'Polyunsaturated_Fat_Lunch', 'Monounsaturated_Fat_Lunch', 'Sodium_(mg)_Lunch',
                   'Trans_Fat_Lunch', 'Cholesterol_Lunch', 'Saturated_Fat_Dinner', 'Polyunsaturated_Fat_Dinner',
                   'Monounsaturated_Fat_Dinner', 'Trans_Fat_Dinner', 'Cholesterol_Dinner', 'Sodium_(mg)_Dinner']

    for i in range(len(fat_columns)):
        if i <= 5:
            fat_breakdown[fat_columns[i]] = nutrient_breakdown[fat_columns[i]] * \
                fat_multiplier * percent_breakfast
        elif i <= 11:
            fat_breakdown[fat_columns[i]] = nutrient_breakdown[fat_columns[i]] * \
                fat_multiplier * percent_lunch
        else:
            fat_breakdown[fat_columns[i]] = nutrient_breakdown[fat_columns[i]] * \
                fat_multiplier * percent_dinner

    protein_breakdown = {}

    protein_columns = ['Protein_(g)_Breakfast', 'Protein_(g)_Lunch', 'Protein_(g)_Dinner']

    protein_breakdown['Protein_(g)_Breakfast'] = nutrient_breakdown['Protein_(g)_Breakfast'] * \
        protein_multiplier * percent_breakfast
    protein_breakdown['Protein_(g)_Lunch'] = nutrient_breakdown['Protein_(g)_Lunch'] * \
        protein_multiplier * percent_lunch
    protein_breakdown['Protein_(g)_Dinner'] = nutrient_breakdown['Protein_(g)_Dinner'] * \
        protein_multiplier * percent_dinner

    carb_breakdown = {}

    carb_columns = ['Carbohydrates_(g)_Breakfast', 'Sugar_Breakfast', 'Carbohydrates_(g)_Lunch',
                    'Sugar_Lunch', 'Carbohydrates_(g)_Dinner', 'Sugar_Dinner']

    for i in range(len(carb_columns)):
        if i <= 1:
            carb_breakdown[carb_columns[i]] = nutrient_breakdown[carb_columns[i]] * \
                carb_multiplier * percent_breakfast
        elif i <= 3:
            carb_breakdown[carb_columns[i]] = nutrient_breakdown[carb_columns[i]] * \
                carb_multiplier * percent_lunch
        else:
            carb_breakdown[carb_columns[i]] = nutrient_breakdown[carb_columns[i]] * \
                carb_multiplier * percent_dinner

    calories_breakdown = {}

    calories_columns = ['Calories_Breakfast', 'Calories_Lunch', 'Calories_Dinner']

    calories_breakdown['Calories_Breakfast'] = total_calories * percent_breakfast
    calories_breakdown['Calories_Lunch'] = total_calories * percent_lunch
    calories_breakdown['Calories_Dinner'] = total_calories * percent_dinner

    return fat_breakdown, protein_breakdown, carb_breakdown, calories_breakdown

In [3]:
def get_features(converted_data, fat_breakdown, protein_breakdown, carb_breakdown, calorie_breakdown):
    minutes_sedentary = (1440 - converted_data['Minutes_Lightly_Active'] -
                         converted_data['Minutes_Fairly_Active'] - converted_data['Minutes_Very_Active'])

    features = {
        'Steps': converted_data['Steps'],
        'Minutes_Sedentary': minutes_sedentary,
        'Minutes_Lightly_Active': converted_data['Minutes_Lightly_Active'],
        'Minutes_Fairly_Active': converted_data['Minutes_Fairly_Active'],
        'Minutes_Very_Active': converted_data['Minutes_Very_Active'],
        'Calories_Breakfast': calorie_breakdown['Calories_Breakfast'],
        'Saturated_Fat_Breakfast': fat_breakdown['Saturated_Fat_Breakfast'],
        'Cholesterol_Breakfast': fat_breakdown['Cholesterol_Breakfast'],
        'Polyunsaturated_Fat_Breakfast': fat_breakdown['Polyunsaturated_Fat_Breakfast'],
        'Monounsaturated_Fat_Breakfast': fat_breakdown['Monounsaturated_Fat_Breakfast'],
        'Trans_Fat_Breakfast': fat_breakdown['Trans_Fat_Breakfast'],
        'Carbohydrates_(g)_Breakfast': carb_breakdown['Carbohydrates_(g)_Breakfast'],
        'Sugar_Breakfast': carb_breakdown['Sugar_Breakfast'],
        'Sodium_(mg)_Breakfast': fat_breakdown['Sodium_(mg)_Breakfast'],
        'Protein_(g)_Breakfast': protein_breakdown['Protein_(g)_Breakfast'],
        'Calories_Lunch': calorie_breakdown['Calories_Lunch'],
        'Saturated_Fat_Lunch': fat_breakdown['Saturated_Fat_Lunch'],
        'Polyunsaturated_Fat_Lunch': fat_breakdown['Polyunsaturated_Fat_Lunch'],
        'Monounsaturated_Fat_Lunch': fat_breakdown['Monounsaturated_Fat_Lunch'],
        'Trans_Fat_Lunch': fat_breakdown['Trans_Fat_Lunch'],
        'Cholesterol_Lunch': fat_breakdown['Cholesterol_Lunch'],
        'Sodium_(mg)_Lunch': fat_breakdown['Sodium_(mg)_Lunch'],
        'Carbohydrates_(g)_Lunch': carb_breakdown['Carbohydrates_(g)_Lunch'],
        'Sugar_Lunch': carb_breakdown['Sugar_Lunch'],
        'Protein_(g)_Lunch': protein_breakdown['Protein_(g)_Lunch'],
        'Calories_Dinner': calorie_breakdown['Calories_Dinner'],
        'Saturated_Fat_Dinner': fat_breakdown['Saturated_Fat_Dinner'],
        'Polyunsaturated_Fat_Dinner': fat_breakdown['Polyunsaturated_Fat_Dinner'],
        'Monounsaturated_Fat_Dinner': fat_breakdown['Monounsaturated_Fat_Dinner'],
        'Trans_Fat_Dinner': fat_breakdown['Trans_Fat_Dinner'],
        'Cholesterol_Dinner': fat_breakdown['Cholesterol_Dinner'],
        'Sodium_(mg)_Dinner': fat_breakdown['Sodium_(mg)_Dinner'],
        'Carbohydrates_(g)_Dinner': carb_breakdown['Carbohydrates_(g)_Dinner'],
        'Sugar_Dinner': carb_breakdown['Sugar_Dinner'],
        'Protein_(g)_Dinner': protein_breakdown['Protein_(g)_Dinner']
    }

    return features

In [109]:
converted_data = {
  'Steps': 10000,
  'Minutes_Lightly_Active': 100,
  'Minutes_Fairly_Active': 60,
  'Minutes_Very_Active': 20,
  'Daily_Calories': 2000,
  'Percent_Breakfast': 20,
  'Percent_Lunch': 40,
  'Percent_Dinner': 40,
  'Daily_Fat': 35,
  'Daily_Carbs': 25,
  'Daily_Protein': 40,
  'Date': '2018-10-01',
  'american': 1.0,
  'italian': 0.0,
  'chinese': 0.0,
  'mexican': 0.0,
  'japanese': 1.0,
  'greek': 1.0,
  'mediterranean': 0.0,
  'barbecue': 0.0,
  'german': 0.0
}

fat_breakdown, protein_breakdown, carb_breakdown, calorie_breakdown = nutritional_breakdown(
        df, converted_data)

fat_breakdown

{'Cholesterol_Breakfast': 47.369361732230225,
 'Cholesterol_Dinner': 59.96923896477862,
 'Cholesterol_Lunch': 52.72309612980455,
 'Monounsaturated_Fat_Breakfast': 1.0669980021649492,
 'Monounsaturated_Fat_Dinner': 1.7396704462301207,
 'Monounsaturated_Fat_Lunch': 1.6779763911159176,
 'Polyunsaturated_Fat_Breakfast': 0.44931076775543205,
 'Polyunsaturated_Fat_Dinner': 0.8091658363152215,
 'Polyunsaturated_Fat_Lunch': 1.3285957268035444,
 'Saturated_Fat_Breakfast': 1.7163035040532133,
 'Saturated_Fat_Dinner': 4.683843656475089,
 'Saturated_Fat_Lunch': 3.845697006801862,
 'Sodium_(mg)_Breakfast': 189.10818245528924,
 'Sodium_(mg)_Dinner': 583.7117425390015,
 'Sodium_(mg)_Lunch': 623.8489537824153,
 'Trans_Fat_Breakfast': 0.057035880857512405,
 'Trans_Fat_Dinner': 0.15129029103265051,
 'Trans_Fat_Lunch': 0.036933843098355235}

In [110]:
features = get_features(converted_data, fat_breakdown, protein_breakdown,
                            carb_breakdown, calorie_breakdown)

features

{'Calories_Breakfast': 400.0,
 'Calories_Dinner': 800.0,
 'Calories_Lunch': 800.0,
 'Carbohydrates_(g)_Breakfast': 7.622350343519052,
 'Carbohydrates_(g)_Dinner': 23.595488384569848,
 'Carbohydrates_(g)_Lunch': 24.440668196871435,
 'Cholesterol_Breakfast': 47.369361732230225,
 'Cholesterol_Dinner': 59.96923896477862,
 'Cholesterol_Lunch': 52.72309612980455,
 'Minutes_Fairly_Active': 60,
 'Minutes_Lightly_Active': 100,
 'Minutes_Sedentary': 1260,
 'Minutes_Very_Active': 20,
 'Monounsaturated_Fat_Breakfast': 1.0669980021649492,
 'Monounsaturated_Fat_Dinner': 1.7396704462301207,
 'Monounsaturated_Fat_Lunch': 1.6779763911159176,
 'Polyunsaturated_Fat_Breakfast': 0.44931076775543205,
 'Polyunsaturated_Fat_Dinner': 0.8091658363152215,
 'Polyunsaturated_Fat_Lunch': 1.3285957268035444,
 'Protein_(g)_Breakfast': 11.091942365553132,
 'Protein_(g)_Dinner': 42.45146186529735,
 'Protein_(g)_Lunch': 39.57893389150156,
 'Saturated_Fat_Breakfast': 1.7163035040532133,
 'Saturated_Fat_Dinner': 4.6838436

In [111]:
def prophet_forecast_weight(features, df, periods, date, cap=None, floor=170):
    future = prophet_final.make_future_dataframe(periods=periods)
    future['floor'] = 170
    
    steps_regressors_df = df[['Date','Steps','Minutes_Sedentary', 'Minutes_Lightly_Active',
       'Minutes_Fairly_Active', 'Minutes_Very_Active', 'Calories_Breakfast',
       'Saturated_Fat_Breakfast', 'Cholesterol_Breakfast',
       'Polyunsaturated_Fat_Breakfast', 'Monounsaturated_Fat_Breakfast',
       'Trans_Fat_Breakfast', 'Carbohydrates_(g)_Breakfast', 'Sugar_Breakfast',
       'Sodium_(mg)_Breakfast',
       'Protein_(g)_Breakfast', 'Calories_Lunch', 'Saturated_Fat_Lunch',
       'Polyunsaturated_Fat_Lunch', 'Monounsaturated_Fat_Lunch',
       'Trans_Fat_Lunch', 'Cholesterol_Lunch', 'Sodium_(mg)_Lunch',
       'Carbohydrates_(g)_Lunch', 'Sugar_Lunch', 'Protein_(g)_Lunch',
       'Calories_Dinner', 'Saturated_Fat_Dinner', 'Polyunsaturated_Fat_Dinner',
       'Monounsaturated_Fat_Dinner', 'Trans_Fat_Dinner', 'Cholesterol_Dinner',
       'Sodium_(mg)_Dinner', 'Carbohydrates_(g)_Dinner',
       'Sugar_Dinner', 'Protein_(g)_Dinner']]
    
    columns = ['Steps','Minutes_Sedentary', 'Minutes_Lightly_Active',
       'Minutes_Fairly_Active', 'Minutes_Very_Active', 'Calories_Breakfast',
       'Saturated_Fat_Breakfast', 'Cholesterol_Breakfast',
       'Polyunsaturated_Fat_Breakfast', 'Monounsaturated_Fat_Breakfast',
       'Trans_Fat_Breakfast', 'Carbohydrates_(g)_Breakfast', 'Sugar_Breakfast',
       'Sodium_(mg)_Breakfast',
       'Protein_(g)_Breakfast', 'Calories_Lunch', 'Saturated_Fat_Lunch',
       'Polyunsaturated_Fat_Lunch', 'Monounsaturated_Fat_Lunch',
       'Trans_Fat_Lunch', 'Cholesterol_Lunch', 'Sodium_(mg)_Lunch',
       'Carbohydrates_(g)_Lunch', 'Sugar_Lunch', 'Protein_(g)_Lunch',
       'Calories_Dinner', 'Saturated_Fat_Dinner', 'Polyunsaturated_Fat_Dinner',
       'Monounsaturated_Fat_Dinner', 'Trans_Fat_Dinner', 'Cholesterol_Dinner',
       'Sodium_(mg)_Dinner', 'Carbohydrates_(g)_Dinner',
       'Sugar_Dinner', 'Protein_(g)_Dinner']

    future = future.merge(steps_regressors_df, left_on='ds', right_on='Date', how='left')
    
    for i in columns:
        future[i].iloc[-periods:] = features[i]
        
    forecast = prophet_final.predict(future)
    
    predicted_weight = np.round(forecast[forecast['ds'] == date]['yhat'].values[0],2)
    predicted_weight_upper = np.round(forecast[forecast['ds'] == date]['yhat_upper'].values[0],2)
    predicted_weight_lower = np.round(forecast[forecast['ds'] == date]['yhat_lower'].values[0],2)
    
    result = {
        
        'predicted_weight':predicted_weight,
        'predicted_weight_upper': predicted_weight_upper,
        'predicted_weight_lower': predicted_weight_lower
        
    }
        
    return result

In [112]:
prophet_forecast_weight(features, df, periods=90, date=converted_data['Date'])

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


{'predicted_weight': 171.21,
 'predicted_weight_lower': 168.46,
 'predicted_weight_upper': 173.87}

In [122]:
def recipe_cosine_similarity(ideal_recipe_vec, new_recipe_vec):
    ## ideal_recipe_vec == array of [ideal_meal_cals, ideal_meal_fat, ideal_meal_protein, ideal_meal_carb]
    ## new_recipe_vec == array of [new_meal_cals, new_meal_fat, new_meal_protein, new_meal_carb]
    return np.dot(ideal_recipe_vec, new_recipe_vec) / np.sqrt(np.dot(ideal_recipe_vec, ideal_recipe_vec) * np.dot(new_recipe_vec, new_recipe_vec))

def get_closest_recipes(ideal_recipe_vec, new_recipe_vecs):
    recipe_similarities = []
    for i in new_recipe_vecs:
        new_recipe_vec = i[2]
        cosine_sim = recipe_cosine_similarity(ideal_recipe_vec, new_recipe_vec)
        if np.isnan(cosine_sim) == False:
            recipe_similarities.append([i[1], i[3], i[4], cosine_sim])
    
    recipe_similarities = sorted(recipe_similarities, key=lambda x: x[3], reverse=True)
    
    return recipe_similarities[:5]

def food_recommendations(df, converted_data):
    
    cuisine_list = [key for key, value in converted_data.items() if value != 0.0]
    
    breakfast_df = df[df['course'].str.contains('breakfast') & df['cuisine'].str.contains('|'.join(cuisine_list))]
    lunch_df = df[df['course'].str.contains('lunch') & df['cuisine'].str.contains('|'.join(cuisine_list))]
    dinner_df = df[df['course'].str.contains('main dishes') & df['cuisine'].str.contains('|'.join(cuisine_list))]
    
    total_calories = converted_data['Daily_Calories']
    percent_fat = converted_data['Daily_Fat']
    percent_protein = converted_data['Daily_Protein']
    percent_carb = converted_data['Daily_Carbs']
    percent_breakfast = converted_data['Percent_Breakfast']
    percent_lunch = converted_data['Percent_Lunch']
    percent_dinner = converted_data['Percent_Dinner']
    
    cals_per_g_fat = 9
    cals_per_g_protein = 4
    cals_per_g_carb = 4

    percent_breakfast = percent_breakfast / 100
    percent_lunch = percent_lunch / 100
    percent_dinner = percent_dinner / 100
    
    calories_breakdown = {}

    calories_columns = ['Calories_Breakfast', 'Calories_Lunch', 'Calories_Dinner']

    calories_breakdown['Calories_Breakfast'] = total_calories * percent_breakfast
    calories_breakdown['Calories_Lunch'] = total_calories * percent_lunch
    calories_breakdown['Calories_Dinner'] = total_calories * percent_dinner

    percent_fat = percent_fat / 100
    percent_protein = percent_protein / 100
    percent_carb = percent_carb / 100
    
    breakfast_recipes = []
    
    g_fat = total_calories * percent_fat / cals_per_g_fat
    g_protein = total_calories * percent_protein / cals_per_g_protein
    g_carbs = total_calories * percent_carb / cals_per_g_carb
    
    g_fat_breakfast = g_fat * percent_breakfast
    g_fat_lunch = g_fat * percent_lunch
    g_fat_dinner = g_fat * percent_dinner
    
    g_protein_breakfast = g_protein * percent_breakfast
    g_protein_lunch = g_protein * percent_lunch
    g_protein_dinner = g_protein * percent_dinner
    
    g_carbs_breakfast = g_carbs * percent_breakfast
    g_carbs_lunch = g_carbs * percent_lunch
    g_carbs_dinner = g_carbs * percent_dinner
    
    ideal_breakfast_recipe_vec = [calories_breakdown['Calories_Breakfast'], g_fat_breakfast, g_protein_breakfast, g_carbs_breakfast]
    ideal_lunch_recipe_vec = [calories_breakdown['Calories_Lunch'], g_fat_lunch, g_protein_lunch, g_carbs_lunch]
    ideal_dinner_recipe_vec = [calories_breakdown['Calories_Dinner'], g_fat_dinner, g_protein_dinner, g_carbs_dinner]
    
    breakfast_recipe_vecs = [(index, row['name'], row[['energy_kcal', 'fat_g', 'protein_g', 'carbs_g']].values, row['source_url'], row['recipe_image']) for index, row in breakfast_df.iterrows()]
    lunch_recipe_vecs = [(index, row['name'], row[['energy_kcal', 'fat_g', 'protein_g', 'carbs_g']].values, row['source_url'], row['recipe_image']) for index, row in lunch_df.iterrows()]
    dinner_recipe_vecs = [(index, row['name'], row[['energy_kcal', 'fat_g', 'protein_g', 'carbs_g']].values, row['source_url'], row['recipe_image']) for index, row in dinner_df.iterrows()]

    breakfast_recipes = get_closest_recipes(ideal_breakfast_recipe_vec, breakfast_recipe_vecs)
    lunch_recipes = get_closest_recipes(ideal_lunch_recipe_vec, lunch_recipe_vecs)
    dinner_recipes = get_closest_recipes(ideal_dinner_recipe_vec, dinner_recipe_vecs)

    return breakfast_recipes, lunch_recipes, dinner_recipes   

In [123]:
yummly_df = pd.read_pickle("Data/Yummly_Data/Yummly_DF.pkl")

In [124]:
# cuisine_list = ['american', 'japanese', 'greek']

In [125]:
food_recommendations(yummly_df, converted_data)

([['Japanese Omelet- Tamagoyaki',
   'http://comfortfoodinfusion.com/japanese-omelet-tamagoyaki/',
   'https://lh3.googleusercontent.com/lvPn7pUSH87JXwUDit9p5AWeDQMVkkb7gSyZto7ceZn97vUseYw7vCEhE1KB82OCfcA2LqeMS3vmgSIApjO_ig=s360',
   0.9992585832010538],
  ['Japanese Tamago Egg',
   'http://allrecipes.com/recipe/221923/japanese-tamago-egg/',
   'https://lh3.googleusercontent.com/qMZbdgaigM3mzo6lqUn_iYTG_T0QZc2GKGANoT7_xtj2QWY-EZ_O2a21KAVy4y5_r_H-9LfqnRmyk-Pb2qnbkw=s360',
   0.9991628839346135],
  ['Paleo Okonomiyaki (Japanese Pancake)',
   'http://grokgrub.com/2014/12/03/paleo-okonomiyaki-japanese-pancake/',
   'http://lh3.ggpht.com/samkjK3-1m2bv6rIrnMrk2rseYnIld7ML8HWEaOJvmoK5ORi-dgaRZMjh-gFz96HIfJ6FYBcihBItrPp-G1Q=s360',
   0.9986846035015454],
  ['Paleo Okonomiyaki (Japanese Pancake)',
   'http://grokgrub.com/2014/12/03/paleo-okonomiyaki-japanese-pancake/',
   'http://lh3.ggpht.com/samkjK3-1m2bv6rIrnMrk2rseYnIld7ML8HWEaOJvmoK5ORi-dgaRZMjh-gFz96HIfJ6FYBcihBItrPp-G1Q=s360',
   0.99868

In [126]:
breakfast_recipes, lunch_recipes, dinner_recipes = food_recommendations(yummly_df, converted_data)
breakfast_recipe_names = [i[0] for i in breakfast_recipes]
breakfast_recipe_urls = [i[1] for i in breakfast_recipes]
breakfast_recipe_images = [i[2] for i in breakfast_recipes]

lunch_recipe_names = [i[0] for i in lunch_recipes]
lunch_recipe_urls = [i[1] for i in lunch_recipes]
lunch_recipe_images = [i[2] for i in lunch_recipes]

dinner_recipe_names = [i[0] for i in dinner_recipes]
dinner_recipe_urls = [i[1] for i in dinner_recipes]
dinner_recipe_images = [i[2] for i in dinner_recipes]

In [127]:
breakfast_recipe_names

['Japanese Omelet- Tamagoyaki',
 'Japanese Tamago Egg',
 'Paleo Okonomiyaki (Japanese Pancake)',
 'Paleo Okonomiyaki (Japanese Pancake)',
 'Greek Egg White Frittata']

In [129]:
breakfast_recipe_info = [i[0:3] for i in breakfast_recipes]
breakfast_recipe_info

[['Japanese Omelet- Tamagoyaki',
  'http://comfortfoodinfusion.com/japanese-omelet-tamagoyaki/',
  'https://lh3.googleusercontent.com/lvPn7pUSH87JXwUDit9p5AWeDQMVkkb7gSyZto7ceZn97vUseYw7vCEhE1KB82OCfcA2LqeMS3vmgSIApjO_ig=s360'],
 ['Japanese Tamago Egg',
  'http://allrecipes.com/recipe/221923/japanese-tamago-egg/',
  'https://lh3.googleusercontent.com/qMZbdgaigM3mzo6lqUn_iYTG_T0QZc2GKGANoT7_xtj2QWY-EZ_O2a21KAVy4y5_r_H-9LfqnRmyk-Pb2qnbkw=s360'],
 ['Paleo Okonomiyaki (Japanese Pancake)',
  'http://grokgrub.com/2014/12/03/paleo-okonomiyaki-japanese-pancake/',
  'http://lh3.ggpht.com/samkjK3-1m2bv6rIrnMrk2rseYnIld7ML8HWEaOJvmoK5ORi-dgaRZMjh-gFz96HIfJ6FYBcihBItrPp-G1Q=s360'],
 ['Paleo Okonomiyaki (Japanese Pancake)',
  'http://grokgrub.com/2014/12/03/paleo-okonomiyaki-japanese-pancake/',
  'http://lh3.ggpht.com/samkjK3-1m2bv6rIrnMrk2rseYnIld7ML8HWEaOJvmoK5ORi-dgaRZMjh-gFz96HIfJ6FYBcihBItrPp-G1Q=s360'],
 ['Greek Egg White Frittata',
  'http://afitphilosophy.com/2016/07/greek-egg-white-frittat

NameError: name 'lunch_recipe_info' is not defined

In [134]:
def prophet_forecast_weight(features, df, periods, date, cap=None, floor=170):
    future = prophet_final.make_future_dataframe(periods=periods)
    future['floor'] = 170
    
    steps_regressors_df = df[['Date','Steps','Minutes_Sedentary', 'Minutes_Lightly_Active',
       'Minutes_Fairly_Active', 'Minutes_Very_Active', 'Calories_Breakfast',
       'Saturated_Fat_Breakfast', 'Cholesterol_Breakfast',
       'Polyunsaturated_Fat_Breakfast', 'Monounsaturated_Fat_Breakfast',
       'Trans_Fat_Breakfast', 'Carbohydrates_(g)_Breakfast', 'Sugar_Breakfast',
       'Sodium_(mg)_Breakfast',
       'Protein_(g)_Breakfast', 'Calories_Lunch', 'Saturated_Fat_Lunch',
       'Polyunsaturated_Fat_Lunch', 'Monounsaturated_Fat_Lunch',
       'Trans_Fat_Lunch', 'Cholesterol_Lunch', 'Sodium_(mg)_Lunch',
       'Carbohydrates_(g)_Lunch', 'Sugar_Lunch', 'Protein_(g)_Lunch',
       'Calories_Dinner', 'Saturated_Fat_Dinner', 'Polyunsaturated_Fat_Dinner',
       'Monounsaturated_Fat_Dinner', 'Trans_Fat_Dinner', 'Cholesterol_Dinner',
       'Sodium_(mg)_Dinner', 'Carbohydrates_(g)_Dinner',
       'Sugar_Dinner', 'Protein_(g)_Dinner']]
    
    columns = ['Steps','Minutes_Sedentary', 'Minutes_Lightly_Active',
       'Minutes_Fairly_Active', 'Minutes_Very_Active', 'Calories_Breakfast',
       'Saturated_Fat_Breakfast', 'Cholesterol_Breakfast',
       'Polyunsaturated_Fat_Breakfast', 'Monounsaturated_Fat_Breakfast',
       'Trans_Fat_Breakfast', 'Carbohydrates_(g)_Breakfast', 'Sugar_Breakfast',
       'Sodium_(mg)_Breakfast',
       'Protein_(g)_Breakfast', 'Calories_Lunch', 'Saturated_Fat_Lunch',
       'Polyunsaturated_Fat_Lunch', 'Monounsaturated_Fat_Lunch',
       'Trans_Fat_Lunch', 'Cholesterol_Lunch', 'Sodium_(mg)_Lunch',
       'Carbohydrates_(g)_Lunch', 'Sugar_Lunch', 'Protein_(g)_Lunch',
       'Calories_Dinner', 'Saturated_Fat_Dinner', 'Polyunsaturated_Fat_Dinner',
       'Monounsaturated_Fat_Dinner', 'Trans_Fat_Dinner', 'Cholesterol_Dinner',
       'Sodium_(mg)_Dinner', 'Carbohydrates_(g)_Dinner',
       'Sugar_Dinner', 'Protein_(g)_Dinner']

    future = future.merge(steps_regressors_df, left_on='ds', right_on='Date', how='left')
    
    for i in columns:
        future[i].iloc[-periods:] = features[i]
        
    forecast = prophet_final.predict(future)
    
    predicted_weight = np.round(forecast[forecast['ds'] == date]['yhat'].values[0],2)
    predicted_weight_upper = np.round(forecast[forecast['ds'] == date]['yhat_upper'].values[0],2)
    predicted_weight_lower = np.round(forecast[forecast['ds'] == date]['yhat_lower'].values[0],2)
    
    result = {
        
        'predicted_weight':predicted_weight,
        'predicted_weight_upper': predicted_weight_upper,
        'predicted_weight_lower': predicted_weight_lower
        
    }
        
    return predicted_weight, predicted_weight_upper, predicted_weight_lower

In [136]:
def flask_output(features, df, yummly_df, periods=90, date=converted_data['Date']):
    
    predicted_weight, predicted_weight_upper, predicted_weight_lower = prophet_forecast_weight(features, df, periods=90, date=converted_data['Date'])
    
    breakfast_recipes, lunch_recipes, dinner_recipes = food_recommendations(yummly_df, converted_data)
    
    breakfast_recipe_names = [i[0] for i in breakfast_recipes]
    lunch_recipe_names = [i[0] for i in lunch_recipes]
    dinner_recipe_names = [i[0] for i in dinner_recipes]
    
    breakfast_recipe_urls = [i[1] for i in breakfast_recipes]
    lunch_recipe_urls = [i[1] for i in lunch_recipes]
    dinner_recipe_urls = [i[1] for i in dinner_recipes]
    
    first_breakfast_recipe_name, first_breakfast_recipe_url = breakfast_recipe_names[0], breakfast_recipe_urls[0]
    second_breakfast_recipe_name, second_breakfast_recipe_url = breakfast_recipe_names[1], breakfast_recipe_urls[1]
    third_breakfast_recipe_name, third_breakfast_recipe_url = breakfast_recipe_names[2], breakfast_recipe_urls[2]
    fourth_breakfast_recipe_name, fourth_breakfast_recipe_url = breakfast_recipe_names[3], breakfast_recipe_urls[3]
    fifth_breakfast_recipe_name, fifth_breakfast_recipe_url = breakfast_recipe_names[4], breakfast_recipe_urls[4]
    
    first_lunch_recipe_name, first_lunch_recipe_url = lunch_recipe_names[0], lunch_recipe_urls[0]
    second_lunch_recipe_name, second_lunch_recipe_url = lunch_recipe_names[1], lunch_recipe_urls[1]
    third_lunch_recipe_name, third_lunch_recipe_url = lunch_recipe_names[2], lunch_recipe_urls[2]
    fourth_lunch_recipe_name, fourth_lunch_recipe_url = lunch_recipe_names[3], lunch_recipe_urls[3]
    fifth_lunch_recipe_name, fifth_lunch_recipe_url = lunch_recipe_names[4], lunch_recipe_urls[4]
    
    first_dinner_recipe_name, first_dinner_recipe_url = dinner_recipe_names[0], dinner_recipe_urls[0]
    second_dinner_recipe_name, second_dinner_recipe_url = dinner_recipe_names[1], dinner_recipe_urls[1]
    third_dinner_recipe_name, third_dinner_recipe_url = dinner_recipe_names[2], dinner_recipe_urls[2]
    fourth_dinner_recipe_name, fourth_dinner_recipe_url = dinner_recipe_names[3], dinner_recipe_urls[3]
    fifth_dinner_recipe_name, fifth_dinner_recipe_url = dinner_recipe_names[4], dinner_recipe_urls[4]
    
    
    breakfast_image = random.choice([i[2] for i in breakfast_recipes])
    lunch_image = random.choice([i[2] for i in lunch_recipes])
    dinner_image =  random.choice([i[2] for i in dinner_recipes])
    
    
    
    result = {
        
        'predicted_weight': predicted_weight,
        'predicted_weight_upper': predicted_weight_upper,
        'predicted_weight_lower': predicted_weight_lower,
        'breakfast_image': breakfast_image,
        'lunch_image': lunch_image,
        'dinner_image': dinner_image,
        'first_breakfast_recipe_name': first_breakfast_recipe_name,
        'second_breakfast_recipe_name': second_breakfast_recipe_name,
        'third_breakfast_recipe_name': third_breakfast_recipe_name,
        'fourth_breakfast_recipe_name': fourth_breakfast_recipe_name,
        'fifth_breakfast_recipe_name': fifth_breakfast_recipe_name,
        'first_breakfast_recipe_url': first_breakfast_recipe_url,
        'second_breakfast_recipe_url': second_breakfast_recipe_url,
        'third_breakfast_recipe_url': third_breakfast_recipe_url,
        'fourth_breakfast_recipe_url': fourth_breakfast_recipe_url,
        'fifth_breakfast_recipe_url': fifth_breakfast_recipe_url,
        'first_lunch_recipe_name': first_lunch_recipe_name,
        'second_lunch_recipe_name': second_lunch_recipe_name,
        'third_lunch_recipe_name': third_lunch_recipe_name,
        'fourth_lunch_recipe_name': fourth_lunch_recipe_name,
        'fifth_lunch_recipe_name': fifth_lunch_recipe_name,
        'first_lunch_recipe_url': first_lunch_recipe_url,
        'second_lunch_recipe_url': second_lunch_recipe_url,
        'third_lunch_recipe_url': third_lunch_recipe_url,
        'fourth_lunch_recipe_url': fourth_lunch_recipe_url,
        'fifth_lunch_recipe_url': fifth_lunch_recipe_url,
        'first_dinner_recipe_name': first_dinner_recipe_name,
        'second_dinner_recipe_name': second_dinner_recipe_name,
        'third_dinner_recipe_name':third_dinner_recipe_name,
        'fourth_dinner_recipe_name': fourth_dinner_recipe_name,
        'fifth_dinner_recipe_name': fifth_dinner_recipe_name,
        'first_dinner_recipe_url': first_dinner_recipe_url,
        'second_dinner_recipe_url': second_dinner_recipe_url,
        'third_dinner_recipe_url': third_dinner_recipe_url,
        'fourth_dinner_recipe_url': fourth_dinner_recipe_url,
        'fifth_dinner_recipe_url': fifth_dinner_recipe_url
        
    }
    
    return result

In [137]:
flask_output(features, df, yummly_df, periods=90, date=converted_data['Date'])

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


{'breakfast_image': 'https://lh3.googleusercontent.com/qMZbdgaigM3mzo6lqUn_iYTG_T0QZc2GKGANoT7_xtj2QWY-EZ_O2a21KAVy4y5_r_H-9LfqnRmyk-Pb2qnbkw=s360',
 'dinner_image': 'https://lh3.googleusercontent.com/8KAekx8duqjArP4OV4JP8zr4B43NAqBgECRfhU4Ijiq9Y12_nj_uzCKOEsO6se6Pxs1jwHDA0Cu0L8Z6SdhN0A=s360',
 'fifth_breakfast_recipe_name': 'Greek Egg White Frittata',
 'fifth_breakfast_recipe_url': 'http://afitphilosophy.com/2016/07/greek-egg-white-frittata.html',
 'fifth_dinner_recipe_name': 'Southern Fried Catfish',
 'fifth_dinner_recipe_url': 'https://www.lemonsforlulu.com/southern-fried-catfish-recipe/',
 'fifth_lunch_recipe_name': 'Southern California Style Burgers',
 'fifth_lunch_recipe_url': 'https://www.theredheadbaker.com/southern-california-style-burgers/',
 'first_breakfast_recipe_name': 'Japanese Omelet- Tamagoyaki',
 'first_breakfast_recipe_url': 'http://comfortfoodinfusion.com/japanese-omelet-tamagoyaki/',
 'first_dinner_recipe_name': 'Greek Roast Lamb with Potatoes',
 'first_dinner_reci